# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

## Data
Additional datasets are available at http://www.timeseriesclassification.com/. Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

## Train MLP
Expected running time for various datasets, on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset | Training time
:-------| :--------
Adiac   | 18 minutes


In [1]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os
import time

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

fdir = '../data'  
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['Adiac']
for each in flist:
    fname = each
    print('Running dataset', fname)
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = min(x_train.shape[0]/10, 16)
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta()    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    end = time.time()
    print('Training time (in seconds)', end-start)
    #Print the testing results which has the lowest training loss.
    print('Test result with the lowest training loss. Loss and accuracy are -')
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

 



Running dataset Adiac
Train on 390 samples, validate on 391 samples
Epoch 1/5000
390/390 [==============================] - 3s 7ms/step - loss: 3.7112 - acc: 0.0487 - val_loss: 3.5229 - val_acc: 0.0639
Epoch 2/5000
390/390 [==============================] - 0s 544us/step - loss: 3.4844 - acc: 0.0795 - val_loss: 3.3993 - val_acc: 0.1228
Epoch 3/5000
390/390 [==============================] - 0s 540us/step - loss: 3.3814 - acc: 0.1000 - val_loss: 3.3412 - val_acc: 0.1074
Epoch 4/5000
390/390 [==============================] - 0s 539us/step - loss: 3.2970 - acc: 0.1231 - val_loss: 3.5467 - val_acc: 0.0716
Epoch 5/5000
390/390 [==============================] - 0s 539us/step - loss: 3.2592 - acc: 0.1077 - val_loss: 3.3259 - val_acc: 0.1202
Epoch 6/5000
390/390 [==============================] - 0s 547us/step - loss: 3.2148 - acc: 0.1179 - val_loss: 3.2336 - val_acc: 0.1049
Epoch 7/5000
390/390 [==============================] - 0s 537us/step - loss: 3.1476 - acc: 0.1385 - val_loss: 3.1248 

390/390 [==============================] - 0s 543us/step - loss: 1.6319 - acc: 0.4436 - val_loss: 1.9446 - val_acc: 0.3606
Epoch 61/5000
390/390 [==============================] - 0s 532us/step - loss: 1.7789 - acc: 0.3949 - val_loss: 1.9058 - val_acc: 0.3632
Epoch 62/5000
390/390 [==============================] - 0s 543us/step - loss: 1.6637 - acc: 0.4487 - val_loss: 2.0526 - val_acc: 0.3862
Epoch 63/5000
390/390 [==============================] - 0s 539us/step - loss: 1.6226 - acc: 0.4769 - val_loss: 1.8644 - val_acc: 0.4066
Epoch 64/5000
390/390 [==============================] - 0s 533us/step - loss: 1.6255 - acc: 0.4769 - val_loss: 2.2112 - val_acc: 0.3069
Epoch 65/5000
390/390 [==============================] - 0s 530us/step - loss: 1.6269 - acc: 0.4359 - val_loss: 1.7612 - val_acc: 0.4118
Epoch 66/5000
390/390 [==============================] - 0s 541us/step - loss: 1.6509 - acc: 0.4590 - val_loss: 1.7250 - val_acc: 0.4501
Epoch 67/5000
390/390 [==============================] 

Epoch 120/5000
390/390 [==============================] - 0s 541us/step - loss: 1.3621 - acc: 0.4949 - val_loss: 1.7588 - val_acc: 0.4808
Epoch 121/5000
390/390 [==============================] - 0s 535us/step - loss: 1.2529 - acc: 0.5282 - val_loss: 1.9239 - val_acc: 0.4143
Epoch 122/5000
390/390 [==============================] - 0s 540us/step - loss: 1.2852 - acc: 0.5487 - val_loss: 1.5947 - val_acc: 0.5243
Epoch 123/5000
390/390 [==============================] - 0s 543us/step - loss: 1.1790 - acc: 0.5897 - val_loss: 1.6328 - val_acc: 0.5269
Epoch 124/5000
390/390 [==============================] - 0s 537us/step - loss: 1.2426 - acc: 0.5641 - val_loss: 1.8243 - val_acc: 0.4552
Epoch 125/5000
390/390 [==============================] - 0s 531us/step - loss: 1.1998 - acc: 0.5538 - val_loss: 1.9562 - val_acc: 0.4118
Epoch 126/5000
390/390 [==============================] - 0s 545us/step - loss: 1.2713 - acc: 0.5564 - val_loss: 1.7523 - val_acc: 0.5064
Epoch 127/5000
390/390 [==========

390/390 [==============================] - 0s 538us/step - loss: 1.1705 - acc: 0.5949 - val_loss: 1.9845 - val_acc: 0.4604
Epoch 180/5000
390/390 [==============================] - 0s 535us/step - loss: 1.0539 - acc: 0.6077 - val_loss: 1.7593 - val_acc: 0.5320
Epoch 181/5000
390/390 [==============================] - 0s 539us/step - loss: 1.0014 - acc: 0.5872 - val_loss: 2.1819 - val_acc: 0.4271
Epoch 182/5000
390/390 [==============================] - 0s 543us/step - loss: 1.0596 - acc: 0.6179 - val_loss: 1.5838 - val_acc: 0.6189
Epoch 183/5000
390/390 [==============================] - 0s 539us/step - loss: 1.0572 - acc: 0.5949 - val_loss: 1.5236 - val_acc: 0.5882
Epoch 184/5000
390/390 [==============================] - 0s 540us/step - loss: 0.9545 - acc: 0.6692 - val_loss: 1.7370 - val_acc: 0.5269
Epoch 185/5000
390/390 [==============================] - 0s 531us/step - loss: 1.0355 - acc: 0.6410 - val_loss: 2.0316 - val_acc: 0.4604
Epoch 186/5000
390/390 [=========================

390/390 [==============================] - 0s 486us/step - loss: 0.8622 - acc: 0.6846 - val_loss: 2.0881 - val_acc: 0.4629
Epoch 239/5000
390/390 [==============================] - 0s 477us/step - loss: 0.7813 - acc: 0.7051 - val_loss: 2.0533 - val_acc: 0.4834
Epoch 240/5000
390/390 [==============================] - 0s 530us/step - loss: 0.9884 - acc: 0.6564 - val_loss: 1.9519 - val_acc: 0.4987
Epoch 241/5000
390/390 [==============================] - 0s 536us/step - loss: 1.0074 - acc: 0.6436 - val_loss: 1.7373 - val_acc: 0.5448
Epoch 242/5000
390/390 [==============================] - 0s 545us/step - loss: 0.8504 - acc: 0.6974 - val_loss: 1.5649 - val_acc: 0.6010
Epoch 243/5000
390/390 [==============================] - 0s 550us/step - loss: 0.7467 - acc: 0.7205 - val_loss: 1.8285 - val_acc: 0.5345
Epoch 244/5000
390/390 [==============================] - 0s 548us/step - loss: 1.0403 - acc: 0.6231 - val_loss: 1.6513 - val_acc: 0.5857
Epoch 245/5000
390/390 [=========================

...

390/390 [==============================] - 0s 530us/step - loss: 0.0254 - acc: 0.9872 - val_loss: 2.6415 - val_acc: 0.7212
Epoch 4723/5000
390/390 [==============================] - 0s 534us/step - loss: 0.0097 - acc: 0.9974 - val_loss: 2.6448 - val_acc: 0.7238
Epoch 4724/5000
390/390 [==============================] - 0s 531us/step - loss: 0.0237 - acc: 0.9949 - val_loss: 2.6838 - val_acc: 0.7238
Epoch 4725/5000
390/390 [==============================] - 0s 533us/step - loss: 0.0326 - acc: 0.9872 - val_loss: 2.6512 - val_acc: 0.7110
Epoch 4726/5000
390/390 [==============================] - 0s 531us/step - loss: 0.0156 - acc: 0.9923 - val_loss: 2.6898 - val_acc: 0.7059
Epoch 4727/5000
390/390 [==============================] - 0s 535us/step - loss: 0.0092 - acc: 0.9974 - val_loss: 2.6666 - val_acc: 0.7110
Epoch 4728/5000
390/390 [==============================] - 0s 545us/step - loss: 0.0129 - acc: 0.9974 - val_loss: 2.6828 - val_acc: 0.7136
Epoch 4729/5000
390/390 [==================

390/390 [==============================] - 0s 529us/step - loss: 0.0113 - acc: 0.9949 - val_loss: 2.6815 - val_acc: 0.7059
Epoch 4782/5000
390/390 [==============================] - 0s 526us/step - loss: 0.0081 - acc: 0.9974 - val_loss: 2.6677 - val_acc: 0.7110
Epoch 4783/5000
390/390 [==============================] - 0s 529us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 2.6514 - val_acc: 0.7059
Epoch 4784/5000
390/390 [==============================] - 0s 523us/step - loss: 0.0116 - acc: 0.9949 - val_loss: 2.6736 - val_acc: 0.7136
Epoch 4785/5000
390/390 [==============================] - 0s 532us/step - loss: 0.0196 - acc: 0.9897 - val_loss: 2.6756 - val_acc: 0.7008
Epoch 4786/5000
390/390 [==============================] - 0s 542us/step - loss: 0.0103 - acc: 0.9974 - val_loss: 2.6846 - val_acc: 0.7212
Epoch 4787/5000
390/390 [==============================] - 0s 530us/step - loss: 0.0585 - acc: 0.9923 - val_loss: 2.7028 - val_acc: 0.7059
Epoch 4788/5000
390/390 [==================

390/390 [==============================] - 0s 550us/step - loss: 0.0058 - acc: 1.0000 - val_loss: 2.6944 - val_acc: 0.7161
Epoch 4841/5000
390/390 [==============================] - 0s 526us/step - loss: 0.0473 - acc: 0.9872 - val_loss: 2.7272 - val_acc: 0.7008
Epoch 4842/5000
390/390 [==============================] - 0s 530us/step - loss: 0.0345 - acc: 0.9897 - val_loss: 2.7221 - val_acc: 0.7084
Epoch 4843/5000
390/390 [==============================] - 0s 546us/step - loss: 0.0082 - acc: 0.9949 - val_loss: 2.7079 - val_acc: 0.7136
Epoch 4844/5000
390/390 [==============================] - 0s 537us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 2.6969 - val_acc: 0.7136
Epoch 4845/5000
390/390 [==============================] - 0s 539us/step - loss: 0.0205 - acc: 0.9923 - val_loss: 2.7127 - val_acc: 0.7033
Epoch 4846/5000
390/390 [==============================] - 0s 549us/step - loss: 0.0133 - acc: 0.9949 - val_loss: 2.6742 - val_acc: 0.7084
Epoch 4847/5000
390/390 [==================

390/390 [==============================] - 0s 528us/step - loss: 0.0101 - acc: 0.9974 - val_loss: 2.7068 - val_acc: 0.6957
Epoch 4900/5000
390/390 [==============================] - 0s 528us/step - loss: 0.0062 - acc: 0.9974 - val_loss: 2.7406 - val_acc: 0.7110
Epoch 4901/5000
390/390 [==============================] - 0s 533us/step - loss: 0.0258 - acc: 0.9872 - val_loss: 2.6853 - val_acc: 0.7187
Epoch 4902/5000
390/390 [==============================] - 0s 534us/step - loss: 0.0050 - acc: 1.0000 - val_loss: 2.6723 - val_acc: 0.7136
Epoch 4903/5000
390/390 [==============================] - 0s 529us/step - loss: 0.0330 - acc: 0.9923 - val_loss: 2.6547 - val_acc: 0.7033
Epoch 4904/5000
390/390 [==============================] - 0s 530us/step - loss: 0.0035 - acc: 1.0000 - val_loss: 2.6563 - val_acc: 0.6982
Epoch 4905/5000
390/390 [==============================] - 0s 530us/step - loss: 0.0123 - acc: 0.9949 - val_loss: 2.6650 - val_acc: 0.6982
Epoch 4906/5000
390/390 [==================

390/390 [==============================] - 0s 533us/step - loss: 0.0192 - acc: 0.9897 - val_loss: 2.6834 - val_acc: 0.7084
Epoch 4959/5000
390/390 [==============================] - 0s 535us/step - loss: 0.0369 - acc: 0.9846 - val_loss: 2.7027 - val_acc: 0.7110
Epoch 4960/5000
390/390 [==============================] - 0s 535us/step - loss: 0.0235 - acc: 0.9923 - val_loss: 2.7149 - val_acc: 0.7033
Epoch 4961/5000
390/390 [==============================] - 0s 522us/step - loss: 0.0153 - acc: 0.9949 - val_loss: 2.6948 - val_acc: 0.7008
Epoch 4962/5000
390/390 [==============================] - 0s 443us/step - loss: 0.0131 - acc: 0.9949 - val_loss: 2.7092 - val_acc: 0.7059
Epoch 4963/5000
390/390 [==============================] - 0s 455us/step - loss: 0.0104 - acc: 0.9974 - val_loss: 2.7319 - val_acc: 0.7033
Epoch 4964/5000
390/390 [==============================] - 0s 452us/step - loss: 0.0074 - acc: 0.9949 - val_loss: 2.6713 - val_acc: 0.7084
Epoch 4965/5000
390/390 [==================

# Recorded results
## Adiac

|Run |Loss |Accuracy
|:---|:--- | :---
|1   |0.005222544357037315  | 0.6930946294608933
|2   |0.0018498263113997382 |0.7109974427601261